<a href="https://colab.research.google.com/github/JKSmirny/hw_22.big-data-challenge.amazon_reviews.amazone.vine/blob/main/JKS2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
# Find the latest version of spark 3.0  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
spark_version = 'spark-3.0.3'
#spark_version = 'spark-3.<enter version>'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:2 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:3 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Get:6 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Hit:7 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Ign:8 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:9 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:12 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic InRelease
Fetched 88.7 kB in 2s (39.5 kB/s)
Reading package lists... Done


# New Section

In [6]:
!wget https://jdbc.postgresql.org/download/postgresql-42.2.9.jar

--2022-05-22 21:17:11--  https://jdbc.postgresql.org/download/postgresql-42.2.9.jar
Resolving jdbc.postgresql.org (jdbc.postgresql.org)... 72.32.157.228, 2001:4800:3e1:1::228
Connecting to jdbc.postgresql.org (jdbc.postgresql.org)|72.32.157.228|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 914037 (893K) [application/java-archive]
Saving to: ‘postgresql-42.2.9.jar’

postgresql-42.2.9.j 100%[===================>] 892.61K  5.21MB/s    in 0.2s    

2022-05-22 21:17:11 (5.21 MB/s) - ‘postgresql-42.2.9.jar’ saved [914037/914037]



In [23]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("CloudETL").config("spark.driver.extraClassPath","/content/postgresql-42.2.9.jar").getOrCreate()

In [31]:
df3 = pd.read_csv('vine_table.csv')
df3

,review_id,star_rating,helpful_votes,total_votes,vine
0,R32VYUWDIB5LKE,5,0,0,N
1,R3CCMP4EV6HAVL,5,0,0,N
2,R14NAE6UGTVTA2,3,1,3,N
3,R2E7QEWSC6EWFA,4,0,1,N
4,R26E6I47GQRYKR,2,3,5,N
...,...,...,...,...,...
254075,R2A0PPKD8EZGE2,3,0,1,N
254076,R3SUK75AQITF8R,5,1,2,N
254077,RMPD5KQHVUXKX,5,0,0,N
254078,R1OG6QX9RPKRLW,5,0,0,N


In [33]:
df3.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 254080 entries, 0 to 254079
Data columns (total 5 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   review_id      254080 non-null  object
 1   star_rating    254080 non-null  int64 
 2   helpful_votes  254080 non-null  int64 
 3   total_votes    254080 non-null  int64 
 4   vine           254080 non-null  object
dtypes: int64(3), object(2)
memory usage: 9.7+ MB


In [34]:
vine_df = df3[df3['total_votes'] >= 20]
vine_df.head()

,review_id,star_rating,helpful_votes,total_votes,vine
118,R2LX6LJ7BA92OO,1,21,27,N
123,R3VMEW1JWIXV1E,1,2,26,N
231,R8RBQTRZ32CFG,1,0,29,N
431,R2612UZ2GU2K29,5,25,32,N
434,R28X8YI8B00C6S,4,52,63,N


In [36]:
helpful_vine = vine_df.loc[vine_df.helpful_votes / vine_df.total_votes >= 0.5]
helpful_vine.head()

,review_id,star_rating,helpful_votes,total_votes,vine
118,R2LX6LJ7BA92OO,1,21,27,N
431,R2612UZ2GU2K29,5,25,32,N
434,R28X8YI8B00C6S,4,52,63,N
438,R3O3RDGDHHINF2,5,41,53,N
657,R1FSDCDV0XV3X6,4,27,33,N


In [37]:
paid_reviews = helpful_vine[helpful_vine['vine'] == 'Y']
paid_reviews.head()

,review_id,star_rating,helpful_votes,total_votes,vine
60837,R22SUWPP3KRZT6,3,21,26,Y
70434,R3O1MAY03ZF3MP,3,19,20,Y
78932,R30LQPCSI3ELHN,5,18,23,Y
86407,R3QIS7D8004AD2,3,76,80,Y
110221,R3R6CDRZU5U2XW,4,41,47,Y


In [38]:
nei_reviews = helpful_vine[helpful_vine['vine'] == 'N']
nei_reviews.head()

,review_id,star_rating,helpful_votes,total_votes,vine
118,R2LX6LJ7BA92OO,1,21,27,N
431,R2612UZ2GU2K29,5,25,32,N
434,R28X8YI8B00C6S,4,52,63,N
438,R3O3RDGDHHINF2,5,41,53,N
657,R1FSDCDV0XV3X6,4,27,33,N


In [39]:
paid_count = len(paid_reviews)
paid_count

19

In [40]:
nei_count = len(nei_reviews)
nei_count

8332

In [42]:

five_star_paid = len(paid_reviews[paid_reviews['star_rating']==5])
five_star_paid

9

In [51]:
round((five_star_paid/paid_count), 2)

0.47

In [49]:
round(paid_reviews['star_rating'].mean(),2)

4.11

In [43]:
five_star_nei = len(nei_reviews[nei_reviews['star_rating']==5])
five_star_nei

4783

In [52]:
round((five_star_nei/nei_count),2)

0.57

In [48]:
round(nei_reviews['star_rating'].mean(),2)

3.94